# Large Model Inference Testing

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

---

## Overview 

This notebook is a **blueprint** for real-time [large model inference (LMI)](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference.html) testing in SageMaker. This notebook uses [Deep Java Library (djl) Large Model Inference (LMI) engines](https://docs.djl.ai/docs/serving/serving/docs/lmi/conceptual_guide/lmi_engine.html) to deploy the models in SageMaker.

This notebook is driven by [LMI configuration YAML files](#lmi-configuration-file). The `test` object in LMI configuration file defines the testing module interface. The testing module provides a prompt generator class that yields prompts for synchronous request-response tests. The prompt used in each test, the generated text, the request latency (secs), the number of output tokens, and tokens per second are recorded in a multi-line JSON output file. 

This notebook does not run concurrent requests to measure request throughput. If such a test is needed, the SageMaker endpoint created by this notebook can be used with your benchmark client.

When comparing latency across different instance types, keep in mind that the LMI engines maybe different. Ensure consistency in LMI engine configurations, as best as possible, so that latency comparison is meaningful. Inconsistencies among LMI configurations can render comparisons across instance types largely meaningless. Different LMI engines will produce different results for the same model and prompt. Therefore, in addition to comparing latency, compare tokens-per-second.

## Hardware Requirements for Running Notebook

Since we are working with large model inference, we will need to download HuggingFace model snapshots, and upload them to S3 bucket. Configure at least 1000 GB volume with the SageMaker Notebook instance. For SageMaker notebook instance type, `ml.m5.2xlarge`, or larger, is recommended..

## Manually Validate Model Outputs

After deploying a new model, do a trial run and manually validate that generated text is not gibberish. If the model is producing gibberish, try deleting and redeploying the model.

## LMI Configuration File

To deploy a model, you need to define a LMI configuration (YAML) file. Below, we show an example LMI configuration file:

```
huggingface:
  model: "mistralai/Mistral-7B-Instruct-v0.2"
  revision: "41b61a33a2483885c981aa79e0df6b32407ed873"
  download: true
djl:
  engine: "MPI"
  option.entryPoint: "djl_python.deepspeed"
  option.tensor_parallel_degree: 4
  option.model_loading_timeout: 1800
  option.dtype: "fp16"
  option.max_tokens: 2048
  option.task: "text-generation"
sagemaker:
  model:
    name: "mistral-7b-instruct-v0-2-deepspeed"
    container: "containers/deepspeed"
    env: 
      HUGGINGFACE_HUB_CACHE: "/tmp"
      TRANSFORMERS_CACHE: "/tmp"
  endpoint:
    name: "mistral-7b-instruct-v0-2-deepspeed"
    instance_type: "ml.g5.12xlarge"
    initial_instance_count: 1
    variant_name: "test"
    model_data_download_timeout_secs: 1800
    container_startup_health_check_timeout_secs: 1200
test:
  module_name: "prompt_generator"
  module_dir: "modules/inst-semeval2017"
  prompt_generator: "PromptGenerator"
  params: { "do_sample": true, "max_new_tokens": 1024, "top_k": 50 }
  warmup_iters: 1
  max_iters: 10
  output_dir: "output/mistral-7b-instruct-v0.2/deepspeed"
```

Below, we provide a brief explanation for the LMI configuration file:

* The `huggingface` object is optional: Alternatively, `djl.option\.model_id` must be specified. 
    * If you specify `huggingface` object, the `huggingface.name` field is required
    * The `huggingface.revision` is required if `huggingface.download` is `true`.
* The `djl` object contains the content for [DJL serving.properties](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html) for the selected [LMI engine](https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/index.html)
    * If you specify `huggingface` object, `djl.option\.model_id` is computed automatically.
* The `sagemaker.model` object is required 
    * The field `sagemaker.model.image` is optional: Alternatively, you can specify `sagemaker.model.container`  as relative path (w.r.t. to this notebook) to the container build script directory.
    * The field `sagemaker.model.env` is optional
* The `sagemaker.endpoint` object is required
    * In `sagemaker.endpoint` object, only `name` and `instance_type` are required.
* The `test` object is optional, and defines the test interface 
    * The `test.module_dir` is relative path. This path is added to `sys.path`. If there is a `requirements.txt` in this directory, it is installed.
    * The `test.module_name` must be a Python module in `test.module_dir`. This module is dynamically loaded.
    * The `test.prompt_generator` is the name of a class in the `test.module_name` module. An object of this class is dyamically created, and the `__call__` method on the object is called to get the prompt generator for testing.
    * The `test.warmpup_iters` are used to warmup the deployed inference model.
    * The `test.max_iters` limits the number of prompt requests, not including the `test.warmup_iters`.
    * The `test.output_dir` is the relative path where `results.json` testing output file is written. 
    * Each line in `results-*.json` file is a json object with following fields: request `prompt`, request output `text`, and request `latency` in seconds.
### Custom Model Handler Code

You can optionally add a [custom model handler](https://github.com/deepjavalibrary/djl-serving/blob/bc7fdfdcbb66b982522e6bc809b0044fabde69e0/serving/docs/streaming_config.md#custom-modelpy-handler) in the `code` sub-folder, collocated with the LMI configuration file, and it is added to the SageMaker model package.

## Initialize SageMaker Session

Let us specify the `s3_bucket` and `s3_prefix` that we will use throughout the notebook.

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

s3_bucket  =  None # specify bucket, or use default sagemaker bucket, if it exists
s3_prefix = 'lmi-djl' # Large model inference with deep java library

role = get_execution_role() # you may provide a pre-existing role ARN here
print(f"SageMaker Execution Role: {role}")

session = boto3.session.Session()
aws_region = session.region_name
print(f"AWS Region: {aws_region}")

sagemaker_session = sagemaker.session.Session(boto_session=session)

try:
    if s3_bucket is None:
        s3_bucket = sagemaker_session.default_bucket()
    s3_client = boto3.client('s3')
    response = s3_client.get_bucket_location(Bucket=s3_bucket)
    bucket_region = response['LocationConstraint']
    bucket_region = 'us-east-1' if bucket_region is None else bucket_region
    
    print(f"Bucket region: {bucket_region}")
    
    try:
        s3_client.head_object(Bucket=s3_bucket, Key=f"{s3_prefix}/")
    except:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_prefix}/")

    print(f"Using S3 folder: s3://{s3_bucket}/{s3_prefix}/ in this notebook")
except:
    print(f"Access Error: Check if '{s3_bucket}' S3 bucket is in '{aws_region}' region, and {s3_prefix} path exists")

sts = boto3.client("sts")
aws_account_id = sts.get_caller_identity()["Account"]

print(f"AWS Account Id: {aws_account_id}")



## Specify SageMaker LMI configuration

We specify a *SageMaker LMI configuration* YAML file for the model we wish to deploy. For example, we specify [DeepSpeed LMI configuration file for mistral-7b-instruct-v0.2](./examples/deepspeed/mistral-7b-instruct-v0.2/config.yaml), below. You can specify whatever example file you wish to deploy.

**Tip:** 
You may wish to make copies of this notebook, if you would like to work with multiple examples concurrently.

In [ ]:
import yaml
import json

import pathlib
print(f"Current working directory: {pathlib.Path().resolve()}")

config_path="examples/transformers-neuronx/llama2-7b-chat/config-streaming.yaml"
with open(config_path, "r") as mf:
    model_config=yaml.safe_load(mf)

print("\nmodel_config:\n")
print(json.dumps(model_config, indent=2))

assert model_config.get('djl', None), "'djl' object is required"

assert model_config.get('huggingface', None) or model_config['djl'].get('option.model_id', None), \
    "'huggingface'  object or 'djl.option\.model_id' is required"

assert model_config.get('sagemaker', None), "'sagemaker' object is required"
assert model_config['sagemaker'].get('model',None), "'sagemaker.model' is required"
assert model_config['sagemaker'].get('endpoint', None), "'sagemaker.endpoint' is required"



## Maybe Build and Push Model Inference Container Image to ECR

Next, if `sagemaker.model.container` is specified, we build and push the container image to Amazon ECR. After building and pushing the image to ECR, we update the `sagemaker.model.image` with the ECR URI for the image. Either `container` or `image` must be specified in `sagemaker.model`.

**Tip:** After building and pushing the container image to ECR, you may want to update `sagemaker.model.image` field in the LMI configuration file with the ECR URI, and delete the image from your local docker repository. This will free up space to build additional docker images, or you are likely to run of of local disk space. Docker, by default, uses the root partition.

In [ ]:
import sys, os, subprocess, stat

sm_model_config = model_config['sagemaker']['model']
container_path = sm_model_config.get("container", None)

if container_path is not None:
    with open(os.path.join(container_path, "build.log"), "w") as logfile:
            print(f"Building and pushing {container_path} to ECR; see log file: {container_path}/build.log")
            container_build_script = os.path.join(container_path, "build_tools", "build_and_push.sh")

            st = os.stat(container_build_script)
            os.chmod(container_build_script, st.st_mode | stat.S_IEXEC)
            subprocess.check_call([container_build_script, aws_region], stdout=logfile, stderr=subprocess.STDOUT)

            image_tag = !cat {container_path}/build_tools/set_env.sh \
                | grep 'IMAGE_TAG' | sed 's/.*IMAGE_TAG=\(.*\)/\1/'

            image_name = !cat {container_path}/build_tools/set_env.sh \
                | grep 'IMAGE_NAME' | sed 's/.*IMAGE_NAME=\(.*\)/\1/'

            ecr_image_uri=f"{aws_account_id}.dkr.ecr.{aws_region}.amazonaws.com/{image_name[0]}:{image_tag[0]}"

    sm_model_config['image'] = ecr_image_uri
else:
    assert sm_model_config.get('image', None) is not None, "'sagemaker.model.image' or 'sagemaker.model.container' is required"

print(f"Model serving image: {sm_model_config['image']}")



## Maybe Download HuggingFace Model Snapshot

If `huggingface` object is specified, only `huggingface.model` is required. If `huggingface.download` is `true`, the HuggingFace model snapshot with revision `huggingface.revision` is downloaded and uploaded to the `s3_bucket`, and `djl.option\.model_id` is set to the S3 URI of the uploaded HuggingFace model snapshot. 

**Note: You must specify `token` below if `huggingface.download`  is `true`, and the `huggingface.model` requires a HuggingFace token for downloading from the HuggingFace hub.**

In [ ]:
hf_spec = model_config.get('huggingface', None)

if hf_spec:
    hf_model = hf_spec.get('model', None)
    download = hf_spec.get('download', None)
    if download:
        revision = hf_spec.get('revision', None)
        assert revision, "'huggingface.revision' is required if 'download' is 'true'"
        
        token = None # Specify HuggingFace token, if required by model

        s3_model_prefix = f"{s3_prefix}/huggingface/models/{hf_model}/{revision}"  # folder where model checkpoint will go
        print(f"s3_model_prefix: {s3_model_prefix}")

        try:
            s3_client.head_object(Bucket=s3_bucket, Key=f"{s3_model_prefix}/config.json")
            print(f"Skipping download; HuggingFace model already exists at s3://{s3_bucket}/{s3_model_prefix}/")
        except:
            subprocess.check_output(f"pip install huggingface-hub", shell=True, stderr=subprocess.STDOUT)
            from huggingface_hub import snapshot_download
            from tempfile import TemporaryDirectory
            from pathlib import Path

            print(f"Downloading HuggingFace model snapshot: {hf_model}, revision: {revision}")
            with TemporaryDirectory(suffix="model", prefix="hf", dir=".") as cache_dir:
                ignore_patterns = ["*.msgpack", "*.h5"]
                snapshot_download(repo_id=hf_model, 
                    revision=revision, 
                    cache_dir=cache_dir,
                    ignore_patterns=ignore_patterns,
                    token=token)

                local_model_path = Path(cache_dir)
                model_snapshot_path = str(list(local_model_path.glob(f"**/snapshots/{revision}"))[0])
                print(f"model_snapshot_path: {model_snapshot_path}")

                for root, dirs, files in os.walk(model_snapshot_path):
                    for file in files:
                        full_path = os.path.join(root, file)
                        with open(full_path, 'rb') as data:
                            key = f"{s3_model_prefix}/{full_path[len(model_snapshot_path)+1:]}"
                            s3_client.upload_fileobj(data, s3_bucket, key)

        model_s3_url = f"s3://{s3_bucket}/{s3_model_prefix}/"
        model_config['djl']['option.model_id'] = model_s3_url
    else:
        model_config['djl']['option.model_id'] = hf_model
    
    print(json.dumps(model_config, indent=2))
    
    

## Create Model Package

Next, we create the model package TAR ball, and upload it to `s3_bucket`. The model package will be used to define the SageMaker model.

In [ ]:
from tempfile import TemporaryDirectory, NamedTemporaryFile
from pathlib import Path
import glob 
import shutil
import os
import tarfile

sm_model_name = sm_model_config.get('name', None)
assert sm_model_name, "'sagemaker.model.name' is required"

with TemporaryDirectory(suffix="pkg", prefix="model", dir=".") as pkg_dir:

    with open(os.path.join(pkg_dir, "serving.properties"), "w") as props_file:
        for key, value in model_config['djl'].items():
            props_file.write(f"{key}={value}\n")

    code_dir = os.path.join(os.path.dirname(config_path), "code")
    if os.path.isdir(code_dir):
        files = glob.glob(f"{code_dir}/*")

        for file in files:
            if os.path.isdir(file):
                shutil.copytree(file, os.path.join(pkg_dir, os.path.basename(file)))
            else:
                shutil.copy2(file, pkg_dir)

    with NamedTemporaryFile(prefix="model", suffix=".gz") as gz_file:
        with tarfile.open(gz_file.name, "w:gz") as tar:
            tar.add(pkg_dir, arcname="")

        gz_file.seek(0)
        model_pkg_key = f"{s3_prefix}/sagemaker/code/{sm_model_name}/model.tar.gz"
        print(f"Upload model package to s3://{s3_bucket}/{model_pkg_key}")
        s3_client.upload_fileobj(gz_file, s3_bucket, model_pkg_key)
        
        


## Create SageMaker Model

Next, we create the SageMaker model using the model package we just uploaded tp `s3_bucket`.

In [ ]:
import boto3

sm_client = boto3.client("sagemaker")
sm_model_image = sm_model_config.get("image", None)
assert sm_model_image, "'sagemaker.model.image' is required"

sm_model_env = sm_model_config.get("env", {})

try:
    create_model_response = sm_client.create_model(
        ModelName=sm_model_name,
        ExecutionRoleArn=role,
        PrimaryContainer={
            "Image": sm_model_image,
            "ModelDataUrl": f"s3://{s3_bucket}/{model_pkg_key}",
            "Environment": sm_model_env,
        },
    )
    model_arn = create_model_response["ModelArn"]

    print(f"Created Model: {model_arn}")
except Exception as e:
    print(f"Error creating model: {e}")

## Create SageMaker Endpoint Config

Next we create endpoint config for the SageMaker model. 

In [ ]:
sm_endpoint_spec = model_config['sagemaker']['endpoint']
endpoint_name = sm_endpoint_spec.get('name', None)
assert endpoint_name, "'sagemaker.endpoint.name' is required"

variant_name = sm_endpoint_spec.get('variant_name', 'test')

instance_type=sm_endpoint_spec.get('instance_type', None)
assert instance_type, "'sagemaker.endpoint.instance_type' is required"

initial_instance_count=sm_endpoint_spec.get('initial_instance_count', 1)
model_data_download_timeout_secs = sm_endpoint_spec.get('model_data_download_timeout_secs', 1200)
container_startup_health_check_timeout_secs=sm_endpoint_spec.get('container_startup_health_check_timeout_secs', 1200)

production_variant = {
            "VariantName": variant_name,
            "ModelName": sm_model_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": initial_instance_count,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_secs,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_secs,
        }

volume_size_gb = sm_endpoint_spec.get('volume_size_gb', None)
if volume_size_gb:
    production_variant['VolumeSizeInGB'] = volume_size_gb

try:
    endpoint_config_response = sm_client.create_endpoint_config(
        EndpointConfigName=endpoint_name,
        ProductionVariants=[production_variant]
    )
    print(endpoint_config_response)
except Exception as e:
    print(f"Error creating endpoint config: {e}")


## Create SageMaker Endpoint

Next, we create the SageMaker Endpoint.

In [ ]:
try: 
    create_endpoint_response = sm_client.create_endpoint(EndpointName=f"{endpoint_name}", 
                                                     EndpointConfigName=endpoint_name)
    print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")
except Exception as e:
    print(f"Error creating endpoint: {e}")


## Run Testing

Now, we are ready to run our tests, using the test interface. During this step, we install HuggingFace  `transformers` library, download model tokenizer configuration files, and create a tokenizer. Next, we load the test interface Python module, create a prompt generator class object, and use it to drive our test run.

In [ ]:
import boto3
import time
from importlib import import_module
import sys
import json
from time import gmtime, strftime
from generate import generate
from utils import get_tokenizer

sm_runtime_client = boto3.client("runtime.sagemaker")

test_spec = model_config.get('test', None)
if test_spec:
    print("installing transformers package")
    subprocess.check_output(f"pip install transformers", shell=True, stderr=subprocess.STDOUT)
    
    module_name = test_spec.get('module_name', None)
    assert module_name, "'test.module_name' is required"
    
    module_dir = test_spec.get('module_dir', None)
    assert module_name, "'test.module_dir' is required"
    
    prompt_generator = test_spec.get('prompt_generator', None)
    assert prompt_generator, "'test.prompt_generator' is required"
    
    output_dir = test_spec.get('output_dir', None)
    assert output_dir, "'test.output_dir' is required"
    os.makedirs(output_dir, exist_ok=True)
    
    sys.path.append(module_dir)
    
    requirements_path = os.path.join(module_dir, "requirements.txt")
    if os.path.isfile(requirements_path):
        print(f"Installing test module requirements...")
        subprocess.check_output(f"pip install -r {requirements_path}", shell=True, stderr=subprocess.STDOUT)
    
    print(f"Loading test module: {module_name} from {module_dir}")
    mod=import_module(module_name)
    prompt_generator_class = getattr(mod, prompt_generator)
    
    print(f"Creating prompt generator object for class: {prompt_generator_class}")
    prompt_generator = prompt_generator_class()()

    warmup_iters = int(test_spec.get('warmup_iters', 1))
    max_iters = int(test_spec.get('max_iters', 10))
    params = test_spec.get("params", None)
    
    ts = strftime("%Y-%m-%d-%H-%M-%S-GMT", gmtime())
    cumu_time = 0.0
    cumu_tokens = 0
    cumu_ttft = 0.0
    
    tokenizer = get_tokenizer(s3_client, model_config['djl']['option.model_id'])
    
    output_formatter = model_config['djl'].get("option.output_formatter", None)
    rolling_batch = model_config['djl'].get("option.rolling_batch", None)
    streaming_enabled = rolling_batch in [ "auto", "deepspeed", "trtllm" ] and output_formatter in [ "jsonlines"]
                                            
    print(f"Streaming enabled: {streaming_enabled}")
    
    try:
        results_path = os.path.join(output_dir, f"results-{instance_type}-{ts}.json")
        with open(results_path, "w") as results:
            count = 0
            
            print("Start testing...")
            while prompt := next(prompt_generator):
                ttft = None
                start_time = time.time()
                
                text, ttft = generate(sm_runtime_client, endpoint_name, prompt, params=params, stream=streaming_enabled)
                latency = time.time() - start_time
                    
                count += 1
                if count <= warmup_iters:
                    print(f"Warm up iteration: {count} of {warmup_iters}. latency: {latency}, ttft: {ttft}")
                    continue
                
                if ttft:
                    cumu_ttft += ttft
                
                iter_count = count - warmup_iters
                
                cumu_time += latency
                index = text.find(prompt)
                if index != -1:
                    text = text[len(prompt):]
                    
                n_tokens = len(tokenizer.encode(text))
                cumu_tokens += n_tokens
                
                tps = n_tokens/latency
                
                json_obj = {"prompt": prompt, 
                            "text": text, 
                            "n_tokens": n_tokens,
                            "latency": latency, 
                            "tps": tps,
                            "ttft": ttft}
                
                results.write(json.dumps( json_obj )+"\n")   
                avg_latency = cumu_time/iter_count
                avg_tps = cumu_tokens/cumu_time
                avg_tokens = cumu_tokens/iter_count
                avg_ttft = cumu_ttft/iter_count
                
                print(f"Iterations completed: {iter_count} of {max_iters}; avg_tokens: {avg_tokens}, avg_latency: {avg_latency} secs, avg_tps: {avg_tps}, avg_ttft: {avg_ttft}")
                if iter_count >= max_iters:
                    break    
        print(f"Testing completed. Results file: {results_path}")
    except StopIteration:
        pass

    


## Cleanup

This concludes the notebook. Below. we delete the **deployed** SageMaker endpoint, endpoint configuration, and the model. 

In [ ]:
response = sm_client.delete_endpoint(EndpointName=endpoint_name)
print(f"Delete Endpoint response: {response}")

response = sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
print(f"Delete Endpoint Config response: {response}")

response = sm_client.delete_model(ModelName=sm_model_name)
print(f"Delete Model response: {response}")

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)
